In [29]:
import requests
import time
import json
import codecs

#url = "https://www.poetryfoundation.org/poems/browse#page=1&sort_by=recently_added"

headers = {'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_2 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13F69 Safari/601.1',
 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
 'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', 'Accept-Encoding': 'none',
 'Accept-Language': 'en-US,en;q=0.8',
 'Connection': 'keep-alive'}

def poem_write_line(poem):
    div_start = poem.find('<div style="text-indent: -1em; padding-left: 1em;">')
    if(div_start == -1):
        return ""
    div_start += len('<div style="text-indent: -1em; padding-left: 1em;">')
    div_end = poem.find('</div>')
    if(div_start >= div_end):
        return ""
    poem_line = poem[div_start:div_end]
    print(poem_line)
    poem = poem[div_end+len('</div>'):]
    return poem

def poem_crawl(link):
    #f = codecs.open("text1.txt", "w", "utf-8")
    url = link
    code = requests.get(url, headers = headers)
    text = code.text
    
    start = text.find('<div class="o-poem " data-view="PoemView">') + len('<div class="o-poem " data-view="PoemView">')
    poem = text[start:]
    poem_len = len(poem)
    while(poem_len != 0):
        poem = poem_write_line(poem)
        poem_len = len(poem)
    #print(text)
    #f.write(text)
    #f.close()

def poem_page_crawl(page_idx):
    url = "https://www.poetryfoundation.org/ajax/poems?page="+str(page_idx)+"&sort_by=recently_added"
    code = requests.get(url, headers = headers)
    text = code.text
    jsn = json.loads(text)
    entry = jsn['Entries']
    for poem in entry:
        title = poem['title']
        print('title : '+title)
        link = poem['link']
        poem_crawl(link)
        time.sleep(1)
        print("\n")
    

poem_page_crawl(1)
#poem_crawl("http://www.poetryfoundation.org/poems/144775/coqui")


title : Coqu&iacute;
One tiny tree frog<br>
 with big eyes<br>
 sings happily,<br>
 “Kokee! Kokee!”<br>
<br>
 His brother comes to bother.<br>
 Coquí doesn't push him.<br>
 Coquí doesn't bite him.<br>
 Coquí tells him,<br>
 “Kokee-Kee! Kokee-Kee!”<br>
<br>
 Two tiny tree frogs<br>
 with big eyes<br>
 sing happily,<br>
 “Kokee! Kokee!”<br>
<br>


title : Imagined Room
<br>


title : Everyday Grace
It can happen like that:<br>
 meeting at the market,<br>
 buying tires amid the smell<br>
 of rubber, the grating sound<br>
 of jack hammers and drills,<br>
 anywhere we share stories,<br>
 and grace flows between us.<br>
   <br>
 The tire center waiting room<br>
 becomes a healing place<br>
 as one speaks of her husband's<br>
 heart valve replacement, bedsores<br>
 from complications. A man<br>
 speaks of multiple surgeries,<br>
 notes his false appearance<br>
 as strong and healthy.<br>
  <br>
 I share my sister's death<br>
 from breast cancer, her<br>
 youngest only seven.<br>
 A woman rise